In [1]:
import os
import pandas as pd
import numpy as np
import numpy.polynomial.polynomial as nppoly
from data_loader import PortLoader
from analyzer import ReturnAnalyzer

### Data Loading

In [2]:
# from datetime import date
import yfinance as yf
bist100 = yf.Ticker('XU100.IS')
bist50 = yf.Ticker('XU050.IS')

bist100_df = yf.download('XU100.IS', start='2022-01-01')
bist50_df = yf.download('XU050.IS', start='2022-01-01')
bist100_df.tail()

# TO-DO: Find API that have BIST50 Index data.

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-16,5211.799805,5259.000000,5154.600098,5214.299805,5214.299805,6291498000
2022-12-19,5270.600098,5404.000000,5267.700195,5391.899902,5391.899902,7195195300
2022-12-20,5408.799805,5445.899902,5344.299805,5419.000000,5419.000000,6991291700
2022-12-21,5449.299805,5491.100098,5412.700195,5429.100098,5429.100098,5943969000
2022-12-22,5462.180176,5515.669922,5398.430176,5447.229980,5447.229980,0


In [3]:
bist100_df['Daily Return'] = (bist100_df['Close'] / bist100_df['Close'].shift(1)) -1
bist50_df['Daily Return'] = (bist50_df['Close'] / bist50_df['Close'].shift(1)) -1 
bist100_df = bist100_df.dropna()
bist50_df = bist50_df.dropna()
bist100_df.head()
bist100_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2022-12-16,5211.799805,5259.000000,5154.600098,5214.299805,5214.299805,6291498000,0.004914
2022-12-19,5270.600098,5404.000000,5267.700195,5391.899902,5391.899902,7195195300,0.034060
2022-12-20,5408.799805,5445.899902,5344.299805,5419.000000,5419.000000,6991291700,0.005026
2022-12-21,5449.299805,5491.100098,5412.700195,5429.100098,5429.100098,5943969000,0.001864
2022-12-22,5462.180176,5515.669922,5398.430176,5447.229980,5447.229980,0,0.003339


### Parsing Files

In [3]:
os.getcwd()

'c:\\Users\\ConquerV\\Documents\\2022-23Fall\\rsm2013\\quant'

In [4]:
# This can be changed to the data directory for weights
os.chdir('c:/Users/ConquerV/Documents/2022-23Fall/rsm2013/quant/data/weight_1')
cwd = os.getcwd()

# store paths to all data xml files in a list for iteration
w1_path = [os.path.join(cwd, f) for f in os.listdir(cwd)]
os.path.basename(w1_path[0])

'longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_win_l_10_win_s_8_es_l_2_es_s_2.csv'

In [5]:
w1_port = PortLoader(w1_path)
w1_port.port_params.head()

Loading Portfolio Data...
Function 'load_port' executed in 0.0070s
Download Market Benchmark...
[*********************100%***********************]  1 of 1 completed


,port_name,long_short,holding_period,training_weight,proportion of short,scale of returns_long,scale of returns_short
0,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
1,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
2,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
3,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01
4,longs_100_day_param_test_w_1_ar_0_c1_10_cs1_1_...,1,100.0,1,0.0,0.1,0.01


### Portfolio Analysis

In [7]:
def sharpe_ratio(port, rf=0):
    """
    This function returns the daily sharpe ratio of a portfolio
    as (E[R] - rf)/Std(R)
    """
    port_ret = pd.DataFrame(port['Daily Return'].dropna())
    mean = port_ret.mean()
    std = port_ret.std()
    sharpe = (mean - rf)/std
    return float(sharpe)

In [172]:
def jensen_alpha(port, benchmark, rf=0):
    """
    Returns excess return of a portfolio according to
    Rp - (rf + beta x (rm - rf))
    """
    port_ret = port['Daily Return']
    mkt_ret  = benchmark['Daily Return']
    port_beta, port_alpha = np.polyfit(benchmark['Daily Return'], port['Daily Return'], 1)

    port_ret_mean = port_ret.mean()
    mkt_ret_mean = mkt_ret.mean()

    port_alpha = port_ret_mean - (rf + port_beta * (mkt_ret_mean - rf))
    port['Excess Return'] = port['Daily Return'] - benchmark['Daily Return']
    return port_alpha, port_beta

In [175]:
def port_analysis(path_list: list, scales: pd.Series, mkt_port: pd.DataFrame):
    """This performs some basic portfolio analysis"""
    temp = {'Portfolio Beta': [],
            'Sharpe':[],
            'Alpha':[]
            }
    for i in range(len(path_list)):
        filepath = path_list[i]
        filename = os.path.basename(filepath)
        if 'param' in filename and 'long' in filename:
            scale = scales[i]
            port = pd.read_csv(filepath)
            port = port.dropna()
            # port['Daily Return'] = port['Daily Return'].apply(lambda x: x/scale)

            # Benchmark
            start, end = port['Dates'].iloc[0], port['Dates'].iloc[-1]
            mkt_port = mkt_port[mkt_port.index.to_series().between(start, end)]
            mkt_port = mkt_port.dropna()

            # Report Metrics
            port_sharpe = sharpe_ratio(port)
            port_alpha, port_beta = jensen_alpha(port, mkt_port)
            
            temp['Portfolio Beta'].append(port_beta)
            temp['Sharpe'].append(port_sharpe)
            temp['Alpha'].append(port_alpha)

    df_metric = pd.DataFrame(data=temp)
    return df_metric
    

In [6]:
analyzer = ReturnAnalyzer(w1_path)
df_metric = analyzer.evaluate()
df_metric.head()


Loading Portfolio Data...
Function 'load_port' executed in 0.0070s
Download Market Benchmark...
[*********************100%***********************]  1 of 1 completed
Begin Evaluating Returns...
Function 'evaluate' executed in 0.8071s


,Params,Beta,Sharpe,Alpha
0,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.732261,5.431685,0.003550
1,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.742263,5.223902,0.003491
2,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.685482,4.798366,0.002738
3,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.737537,5.588751,0.003966
4,0 longs_100_day_param_test_w_1_ar_0_c1_10...,0.733396,5.581227,0.004067
